In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Xgb, Model1Lgb, Model1NN, Model2Linear


%matplotlib inline

print("OK\n")

OK



In [2]:
def geometric_mean(x):
    return np.exp(np.log(x[x>0]).mean())

In [3]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
#app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
#app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custom data
#app_train_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_loan_from_hc_only_cash.csv')
#app_test_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_loan_from_hc_only_cash.csv')
#app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_annuity_to_max_installment_ratio.csv')
#app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_annuity_to_max_installment_ratio.csv')

In [6]:
#app_train = app_train_origin.copy()
#app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
#app_train_1000_mean = app_train_1000_mean_origin.copy()
#app_test_1000_mean = app_test_1000_mean_origin.copy()
#app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
#app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [7]:
df = pd.read_csv(r'./home-credit-default-risk/application_train.csv')
test_df = pd.read_csv(r'./home-credit-default-risk/application_test.csv')
df = df.append(test_df).reset_index()

docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 列名から不正な文字を削除
for col in df.columns:
    df[col] = df[col].replace('\s', '_', regex=True)
    df[col] = df[col].replace(':', '_', regex=True)
    df[col] = df[col].replace('/', '_', regex=True)


# nullに置き換え
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
df['INCOME_div_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
df['INCOME_div_EMPLOYED'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']

#
df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)

df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)

df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])

df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = df['DAYS_EMPLOYED'] / (df['DAYS_BIRTH'] + 18*365)

df['NEW_BIRTH_TO_EMPLOY_RATIO'] = df['DAYS_BIRTH'] / (1 + df['DAYS_EMPLOYED'])

df['NEW_INCOME_TO_ANNUITY_RATIO'] = df['AMT_INCOME_TOTAL'] / (1 + df['AMT_ANNUITY'])

df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
df['NEW_EXT_SOURCES_MEDIAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

df['EXT_SOURCE_mean'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
df['EXT_SOURCE_max'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

df['EXT_SOURCE_min'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

df['EXT_SOURCE_std'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

df['EXT_SOURCE_count'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

df['NEW_EXT_SOURCES_PROD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

df['NEW_EXT_SOURCES_MAD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

df['NEW_EXT_SOURCES_GEO'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
df['DAYS_EMPLOYED_div_BERTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
df['ANNUITY_div_INCOME'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
df['ANNUITY_div_CREDIT'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

### 
df['credit_goods_price_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
 
df['credit_downpayment'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

df['USER_AGE'] = abs(df['DAYS_BIRTH'])/364.25

df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']

df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']

df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

df['NEW_PHONE_TO_EMPLOYED_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']

df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

### 2022/06/25
df['app_EXT_SOURCE_prod'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_1'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_2'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_3'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = df['EXT_SOURCE_1'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = df['EXT_SOURCE_2'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = df['EXT_SOURCE_3'] / df['DAYS_BIRTH']

### 2022/06/26
df['app_most_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
df['app_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
df['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']
df['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']

### 2022/06/29
#### EXT_SOURCE_X FEATURE
#df['APP_SCORE1_TO_BIRTH_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE2_TO_BIRTH_RATIO'] = df['EXT_SOURCE_2'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE3_TO_BIRTH_RATIO'] = df['EXT_SOURCE_3'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE1_TO_EMPLOY_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_EMPLOYED'] / 365.25)
#### Time ratios
#df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']

### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#### one-hotencoding処理対象列を追加
encoding_columns.append('CODE_GENDER')
encoding_columns.append('FLAG_OWN_CAR')
encoding_columns.append('FLAG_OWN_REALTY')
encoding_columns.append('NAME_INCOME_TYPE')
encoding_columns.append('NAME_FAMILY_STATUS')

#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

### 列名の不正な文字を置換(ドット対応)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 極大値、極小値をNULLへ置換
df = df.replace([np.inf, -np.inf], np.nan)

len(df.columns)

df.head()

NAME_CONTRACT_TYPE
NAME_TYPE_SUITE
NAME_EDUCATION_TYPE
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,1,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,1,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,1,0,0,0,0


In [8]:
### indexを削除
df=df.reset_index()
df.head()

,level_0,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0,0,0,0,0,1,0,0,0,0
1,1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,0,0,0,1,0,0,0,0,0
2,2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0,0,0,0,0,1,0,0,0,0
3,3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0,0,0,0,0,0,1,0,0,0
4,4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0,0,0,0,0,1,0,0,0,0


In [9]:
### indexを削除
df = df.drop(columns=['index'])
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,1,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,1,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,1,0,0,0,0


In [10]:
# infをnanへ置換
bureau = bureau.replace([np.inf, -np.inf], np.nan)
bureau_balance = bureau_balance.replace([np.inf, -np.inf], np.nan)
pos_cash_balance = pos_cash_balance.replace([np.inf, -np.inf], np.nan)
credit_card_balance = credit_card_balance.replace([np.inf, -np.inf], np.nan)
previous_application = previous_application.replace([np.inf, -np.inf], np.nan)
installments_payments = installments_payments.replace([np.inf, -np.inf], np.nan)

pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [11]:
baseline = Baseline('SK_ID_CURR')

pos = pos_cash_balance.copy()
pos = baseline.reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage of optimization is 290039510.00 MB
Decreased by -47513511.5%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [12]:
pos_ohe = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
print(len(col_ohe))
col_ohe

10


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Amortized debt',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CONTRACT_STATUS_nan']

In [13]:
pos_ohe_agg = pos_ohe.groupby('SK_ID_CURR').agg({
    "MONTHS_BALANCE": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT_FUTURE": ["mean", "std", "min", "max"]
    ,"SK_DPD": ["mean", "std", "min", "max"]
    ,"SK_DPD_DEF": ["mean", "std", "min", "max"]
    # カテゴリ変数をone-hot-encodingした値の集約
    ,"NAME_CONTRACT_STATUS_Active": ["mean"]
    ,"NAME_CONTRACT_STATUS_Amortized debt": ["mean"]
    ,"NAME_CONTRACT_STATUS_Approved": ["mean"]
    ,"NAME_CONTRACT_STATUS_Canceled": ["mean"]
    ,"NAME_CONTRACT_STATUS_Completed": ["mean"]
    ,"NAME_CONTRACT_STATUS_Demand": ["mean"]
    ,"NAME_CONTRACT_STATUS_Returned to the store": ["mean"]
    ,"NAME_CONTRACT_STATUS_Signed": ["mean"]
    ,"NAME_CONTRACT_STATUS_XNA": ["mean"]
    ,"NAME_CONTRACT_STATUS_nan": ["mean"]
    # IDのユニーク数をカウント
    ,"SK_ID_PREV":["count", "nunique"]
})

# カラム名の付与
pos_ohe_agg.columns = [i + "_" + j for i,j in pos_ohe_agg.columns]
pos_ohe_agg = pos_ohe_agg.reset_index(drop=False)

print(pos_ohe_agg.shape)
pos_ohe_agg.head()

(337252, 33)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [14]:
### infをnanへ置換
pos_ohe_agg = pos_ohe_agg.replace([np.inf, -np.inf], np.nan)
pos_ohe_agg.head()

,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [15]:
credit = credit_card_balance_origin.copy()
cerdit = baseline.reduce_mem_usage(credit)
print(credit.shape)
credit.head()

Memory usage of dataframe is 673.88 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -49579318.7%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [16]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in credit.columns:
    if credit[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

NAME_CONTRACT_STATUS


['NAME_CONTRACT_STATUS']

In [17]:
# one-hot-encoding
for col in encoding_columns:
    credit_ohe = pd.get_dummies(credit, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(credit_ohe.columns) - set(credit.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [18]:
credit_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'SK_ID_CURR': ['mean', 'std', 'min', 'max']
        ,'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
credit_ohe_agg.columns = [i + "_" + j for i,j in credit_ohe_agg.columns]
credit_ohe_agg = credit_ohe_agg.reset_index(drop=False)

print(credit_ohe_agg.shape)
credit_ohe_agg.head()

(103558, 95)


,SK_ID_CURR,SK_ID_CURR_mean,SK_ID_CURR_std,SK_ID_CURR_min,SK_ID_CURR_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,100006.0,0.0,100006,100006,-3.5,1.870829,-6,-1,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,100011.0,0.0,100011,100011,-38.5,21.505813,-75,-2,54482.109375,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,100013.0,0.0,100013,100013,-48.5,27.856777,-96,-1,18159.919922,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,100021.0,0.0,100021,100021,-10.0,5.049752,-18,-2,0.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,100023.0,0.0,100023,100023,-7.5,2.449490,-11,-4,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [19]:
# infをnanへ置換
credit_ohe_agg = credit_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [20]:
previous = previous_application_origin.copy()
previous = baseline.reduce_mem_usage(credit)
print(previous.shape)
previous.head()

Memory usage of dataframe is 318.63 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -104857500.0%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [21]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in previous.columns:
    if previous[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_STATUS


In [22]:
# one-hot-encoding
for col in encoding_columns:
    previous_ohe = pd.get_dummies(previous, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(previous_ohe.columns) - set(previous.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [23]:
previous_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
previous_ohe_agg.columns = [i + "_" + j for i,j in previous_ohe_agg.columns]
previous_ohe_agg = previous_ohe_agg.reset_index(drop=False)

print(previous_ohe_agg.shape)
previous_ohe_agg.head()

(103558, 91)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_std,AMT_BALANCE_min,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,-3.5,1.870829,-6,-1,0.000000,0.000000,0.0,0.00000,270000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,-38.5,21.505813,-75,-2,54482.109375,68127.238133,0.0,189000.00000,164189.189189,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,-48.5,27.856777,-96,-1,18159.919922,43237.406465,0.0,161420.21875,131718.750000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,-10.0,5.049752,-18,-2,0.000000,0.000000,0.0,0.00000,675000.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,-7.5,2.449490,-11,-4,0.000000,0.000000,0.0,0.00000,135000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [24]:
# infをnanへ置換
previous_ohe_agg = previous_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [25]:
installments = installments_payments_origin.copy()
installments = baseline.reduce_mem_usage(installments)
print(installments.shape)
installments.head()

Memory usage of dataframe is 830.41 MB
Memory usage of optimization is 408162158.00 MB
Decreased by -49151908.2%
(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [26]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in installments.columns:
    if installments[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

[]

In [27]:
# one-hot-encoding
#for col in encoding_columns:
#    installments_ohe = pd.get_dummies(installments, columns=[col], dummy_na=True)
#    col_ohe = sorted(list(set(installments_ohe.columns) - set(installments.columns)))

#print(len(col_ohe))
#col_ohe
installments_ohe = installments.copy()

In [28]:
installments_ohe = installments_ohe.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_IP'})

In [29]:
installments_ohe_agg = installments_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'NUM_INSTALMENT_VERSION': ['mean', 'std', 'min', 'max']
        ,'NUM_INSTALMENT_NUMBER': ['mean', 'std', 'min', 'max']
        ,'DAYS_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENTRY_PAYMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT': ['mean', 'std', 'min', 'max']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV_IP":["count", "nunique"]
    }
)

# カラム名の付与
installments_ohe_agg.columns = [i + "_" + j for i,j in installments_ohe_agg.columns]
installments_ohe_agg = installments_ohe_agg.reset_index(drop=False)

print(installments_ohe_agg.shape)
installments_ohe_agg.head()

(339587, 27)


,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_std,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_NUMBER_mean,NUM_INSTALMENT_NUMBER_std,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_NUMBER_max,DAYS_INSTALMENT_mean,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,100001,1.142857,0.377964,1.0,2.0,2.714286,1.112697,1,4,-2187.714355,...,5885.132324,5076.676763,3951.000000,17397.900391,5885.132324,5076.676763,3951.000000,17397.900391,7,2
1,100002,1.052632,0.229416,1.0,2.0,10.000000,5.627314,1,19,-295.000000,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19,1
2,100003,1.040000,0.200000,1.0,2.0,5.080000,3.134751,1,12,-1378.160034,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25,3
3,100004,1.333333,0.577350,1.0,2.0,2.000000,1.000000,1,3,-754.000000,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3,1
4,100005,1.111111,0.333333,1.0,2.0,5.000000,2.738613,1,9,-586.000000,...,6240.205078,4281.014648,4813.200195,17656.244141,6240.205078,4281.014648,4813.200195,17656.244141,9,1


In [30]:
# infをnanへ置換
installments_ohe_agg = installments_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [31]:
bureau = bureau_origin.copy()
bureau = baseline.reduce_mem_usage(bureau)
print(bureau.shape)
bureau.head()

Memory usage of dataframe is 222.62 MB
Memory usage of optimization is 125299372.00 MB
Decreased by -56283785.5%
(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [32]:
### 
for col in bureau.columns:
    bureau[col] = bureau[col].replace('\s', '_', regex=True)
    bureau[col] = bureau[col].replace(':', '_', regex=True)
    bureau[col] = bureau[col].replace('/', '_', regex=True)
    
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency_1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer_credit,-131,NaN
1,215354,5714463,Active,currency_1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit_card,-20,NaN
2,215354,5714464,Active,currency_1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer_credit,-16,NaN
3,215354,5714465,Active,currency_1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit_card,-16,NaN
4,215354,5714466,Active,currency_1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer_credit,-21,NaN


In [33]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in bureau.columns:
    if bureau[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE


In [34]:
# one-hot-encoding
bureau_ohe = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_CURRENCY'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_TYPE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

print(len(col_ohe))
col_ohe

26


['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad_debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency_1',
 'CREDIT_CURRENCY_currency_2',
 'CREDIT_CURRENCY_currency_3',
 'CREDIT_CURRENCY_currency_4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another_type_of_loan',
 'CREDIT_TYPE_Car_loan',
 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
 'CREDIT_TYPE_Consumer_credit',
 'CREDIT_TYPE_Credit_card',
 'CREDIT_TYPE_Interbank_credit',
 'CREDIT_TYPE_Loan_for_business_development',
 'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
 'CREDIT_TYPE_Loan_for_working_capital_replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile_operator_loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real_estate_loan',
 'CREDIT_TYPE_Unknown_type_of_loan',
 'CREDIT_TYPE_nan']

In [35]:
bureau_ohe = bureau_ohe.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_BUREAU'})

In [36]:
bureau_ohe_agg = bureau_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'DAYS_CREDIT': ['mean', 'std', 'min', 'max', 'var']
        ,'CREDIT_DAY_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_ENDDATE': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENDDATE_FACT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_MAX_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'CNT_CREDIT_PROLONG': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_DEBT': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_LIMIT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_UPDATE': ['mean', 'std', 'min', 'max']
        ,'AMT_ANNUITY': ['mean', 'std', 'min', 'max']
        # 
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        ,'CREDIT_ACTIVE_Active': ['mean']
        ,'CREDIT_ACTIVE_Bad_debt': ['mean']
        ,'CREDIT_ACTIVE_Closed': ['mean']
        ,'CREDIT_ACTIVE_Sold': ['mean']
        ,'CREDIT_ACTIVE_nan': ['mean']
        ,'CREDIT_CURRENCY_currency_1': ['mean']
        ,'CREDIT_CURRENCY_currency_2': ['mean']
        ,'CREDIT_CURRENCY_currency_3': ['mean']
        ,'CREDIT_CURRENCY_currency_4': ['mean']
        ,'CREDIT_CURRENCY_nan': ['mean']
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        # IDのユニーク数をカウント
        ,'SK_ID_BUREAU_BUREAU': ["count", "nunique"]
    }
)

# カラム名の付与
bureau_ohe_agg.columns = [i + "_" + j for i,j in bureau_ohe_agg.columns]
bureau_ohe_agg = bureau_ohe_agg.reset_index(drop=False)

#
bureau_ohe_agg['debt_credit_ratio_None'] = bureau_ohe_agg['AMT_CREDIT_SUM_DEBT_sum'] / bureau_ohe_agg['AMT_CREDIT_SUM_sum']

print(bureau_ohe_agg.shape)
bureau_ohe_agg.head()

(305811, 81)


,SK_ID_CURR,DAYS_CREDIT_mean,DAYS_CREDIT_std,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_var,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,100001,-735.000000,489.942514,-1572,-49,240043.666667,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,7,0.410555
1,100002,-874.000000,431.451040,-1437,-103,186150.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,8,0.284122
2,100003,-1400.750000,909.826128,-2586,-606,827783.583333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,4,0.000000
3,100004,-867.000000,649.124025,-1326,-408,421362.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,2,0.000000
4,100005,-190.666667,162.297053,-373,-62,26340.333333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,0.864992


In [37]:
# consumer_annuity_to_max_installment_ratio
app_train_mean1 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_test_mean1 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_train_mean1 = app_train_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean1 = app_test_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

### installment_payment_ratio_1000_mean_mean
#### consumer_installment_payment_ratio_1000_mean_mean
app_train_mean2 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean2 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean2 = app_train_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean2 = app_test_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


#### revo_installment_payment_ratio_1000_mean_mean
app_train_mean3 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean3 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean3 = app_train_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean3 = app_test_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

#### installment_payment_ratio_1000_mean_mean
app_train_mean4 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean4 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean4 = app_train_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean4 = app_test_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


In [38]:
# infをnanへ置換
app_train_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)
app_test_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)

app_train_mean2 = app_train_mean2.replace([np.inf, -np.inf], np.nan)
app_test_mean2 = app_test_mean2.replace([np.inf, -np.inf], np.nan)

app_train_mean3 = app_train_mean3.replace([np.inf, -np.inf], np.nan)
app_test_mean3 = app_test_mean3.replace([np.inf, -np.inf], np.nan)

app_train_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)
app_test_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)

In [39]:
### bureau_balance
df = df.merge(bureau_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,0.284122
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,4.0,0.000000
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.000000
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000


In [40]:
### previos_application
df = df.merge(previous_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
### pos_cash
df = df.merge(pos_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [42]:
### instllment_payment
df = df.merge(installments_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,62947.085938,168097.620686,2482.919922,691786.875000,62947.085938,168097.620686,2482.919922,691786.875000,16.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,12666.444336,7852.910602,1821.780029,22678.785156,12214.060547,8048.060783,0.180000,22678.785156,66.0,5.0


In [43]:
### credit_card_balance
df = df.merge(credit_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean_y,NAME_CONTRACT_STATUS_Sent proposal_mean_y,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)

In [45]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 636 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 619 features are remained after removing non-informative features
119 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 500 features are remained after removing features not interesting for LightGBM classifier
---=> 36 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 536 features

---=> df final shape: (356250, 536)  <=--- 



,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_high_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,CNT_CREDIT_PROLONG_max_high_risk,CNT_CREDIT_PROLONG_max_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,1,0,1,0,0,0,0,0,1
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,1,0,1,0,0,0,0,0,1
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,1,0,1,0,0,0,0,0,1
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0,1,0,0,1,0,1,0,0,1
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,1,0,1,0,0,0,0,0,1


In [46]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 536 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 535 features are remained after removing non-informative features
41 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 494 features are remained after removing features not interesting for LightGBM classifier
---=> 13 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 507 features

---=> df final shape: (356250, 507)  <=--- 



,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,1,1,1,1,0,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,1,1,1,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,1,1,1,1,0,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0,0,0,0,1,1,0,1,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,1,1,1,1,0,0,0,0,0


In [47]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,1,1,1,1,0,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,1,1,1,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,1,1,1,1,0,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0,0,0,0,1,1,0,1,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,1,1,1,1,0,0,0,0,0


In [48]:
# split train, and test datasets
df_train = df[df['TARGET'].notnull()]
df_test = df[df['TARGET'].isnull()]

print(df_test.shape)
df_test.head()

(48744, 507)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk
307506,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,1,1,1,1,0,0,0,0,0
307507,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,1,1,1,1,0,0,0,0,0
307508,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,0,1,1,1,1,1,0,1,0,1
307509,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,1,1,0,1,0,1,0,1
307510,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0,0,0


In [49]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [50]:
df_train = df_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,1,1,1,0,0,0,0,0,0.040068
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,1,1,1,0,0,0,0,0,0.095240
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,1,1,1,0,1,0,1,0.044435
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,1,1,0,1,0,1,0,1,0.036586
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0,0,0.141754


In [51]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [52]:
df_train = df_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,1,1,0,0,0,0,0,0.040068,0.035234
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,1,1,0,0,0,0,0,0.095240,0.107426
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,1,1,0,1,0,1,0.044435,0.048029
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,1,0,1,0,1,0,1,0.036586,0.032975
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0,0.141754,0.161962


In [53]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [54]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,1,0,0,0,0,0,0.040068,0.035234,0.042769
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,1,0,0,0,0,0,0.095240,0.107426,0.103694
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,1,0,1,0,1,0.044435,0.048029,0.036006
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0,1,0.036586,0.032975,0.048812
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0.141754,0.161962,0.173045


In [55]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [56]:
df_train = df_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,0,0,0,0,0,0.040068,0.035234,0.042769,0.035013
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,0,0,0,0,0,0.095240,0.107426,0.103694,0.103532
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,0,1,0,1,0.044435,0.048029,0.036006,0.048415
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,0,1,0,1,0.036586,0.032975,0.048812,0.032249
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0.141754,0.161962,0.173045,0.150442


In [57]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [58]:
df_train = df_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637


In [59]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [60]:
df_train = df_train.merge(train_set3a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set3a, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,0,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575


In [61]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [62]:
df_train = df_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [63]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,0,0,0.260928,0.199398,0.242499,0.198794,0.240473,0.223662,0.259405
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,0,0,0.012171,0.012061,0.017857,0.012648,0.014736,0.024137,0.023080
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,0,0,0.039950,0.047440,0.033203,0.033060,0.029793,0.035314,0.033290
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,1,0,0,0.030776,0.030071,0.048900,0.025442,0.031804,0.031135,0.022860
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,0,0,0.035088,0.023555,0.043556,0.039186,0.060946,0.035167,0.033131


In [64]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [65]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [66]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,0,0.260928,0.199398,0.242499,0.198794,0.240473,0.223662,0.259405
100003,1,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,0,0.012171,0.012061,0.017857,0.012648,0.014736,0.024137,0.023080
100004,2,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0.039950,0.047440,0.033203,0.033060,0.029793,0.035314,0.033290
100006,3,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,1,0,0,0.030776,0.030071,0.048900,0.025442,0.031804,0.031135,0.022860
100007,4,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,0,0.035088,0.023555,0.043556,0.039186,0.060946,0.035167,0.033131


In [67]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-1.732065,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,-0.627631,0.0,-0.502076,1.896448,1.301762,1.758129,1.262144,1.746196,1.488601,1.888223
100003,-1.732054,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,-0.627631,0.0,-0.502076,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.588371,-0.587171
100004,-1.732043,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,-0.627631,0.0,-0.502076,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.472024,-0.480227
100006,-1.732031,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,1.593292,0.0,-0.502076,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.515528,-0.589469
100007,-1.732020,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,-0.627631,0.0,-0.502076,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.473551,-0.481891


In [68]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100002,-1.732065,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,...,-0.627631,0.0,-0.502076,1.896448,1.301762,1.758129,1.262144,1.746196,1.488601,1.888223
1,100003,-1.732054,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,...,-0.627631,0.0,-0.502076,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.588371,-0.587171
2,100004,-1.732043,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,...,-0.627631,0.0,-0.502076,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.472024,-0.480227
3,100006,-1.732031,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,...,1.593292,0.0,-0.502076,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.515528,-0.589469
4,100007,-1.732020,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,...,-0.627631,0.0,-0.502076,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.473551,-0.481891


In [69]:
### level_0を削除
df_train.drop('level_0', axis=1, inplace=True)
df_test.drop('level_0', axis=1, inplace=True)
df_test.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,NaN,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,-0.146484,-0.734184,0.024017,...,-0.627631,0.0,-0.502076,-0.419358,-0.450892,-0.402589,-0.457814,-0.317197,-0.328307,-0.469542
1,100005,NaN,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,1.079102,-0.464476,-0.891113,...,-0.627631,0.0,-0.502076,0.159147,0.319855,0.256510,0.261749,0.678170,0.365058,0.507163
2,100013,NaN,-0.577531,0.313221,0.159594,2.943903,0.247943,-0.127563,-0.916815,-0.886230,...,1.593292,0.0,1.991729,-0.373564,-0.314285,-0.475758,-0.317065,-0.542220,-0.171688,-0.358904
3,100028,NaN,2.192099,1.347112,2.424818,1.511669,2.805819,0.399658,0.472284,0.221558,...,1.593292,0.0,1.991729,-0.455868,-0.475006,-0.337219,-0.486834,-0.424040,-0.434369,-0.505095
4,100038,NaN,0.807284,0.106443,0.065768,0.342100,0.235763,-0.783203,0.686768,0.082153,...,-0.627631,0.0,-0.502076,0.646859,0.902093,1.006761,0.754380,1.230772,1.196224,1.279548


In [70]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 636.96 MB
Memory usage of optimization is 631002440.00 MB
Decreased by -99064263.5%
Memory usage of dataframe is 100.97 MB
Memory usage of optimization is 100217792.00 MB
Decreased by -99257377.1%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100002,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,...,-0.627441,0.0,-0.501953,1.896484,1.301758,1.757812,1.261719,1.746094,1.488281,1.888672
1,100003,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,...,-0.627441,0.0,-0.501953,-0.711914,-0.698242,-0.671875,-0.692871,-0.686035,-0.588379,-0.587402
2,100004,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,...,-0.627441,0.0,-0.501953,-0.420654,-0.320557,-0.505859,-0.478271,-0.523926,-0.471924,-0.480225
3,100006,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,...,1.593750,0.0,-0.501953,-0.516602,-0.505859,-0.336182,-0.558105,-0.501953,-0.515625,-0.589355
4,100007,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,...,-0.627441,0.0,-0.501953,-0.471680,-0.575684,-0.394043,-0.414062,-0.188232,-0.473633,-0.481934


In [71]:
df_test.drop('TARGET', axis = 1, inplace = True)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,-0.577637,-0.307129,-0.075134,-0.451904,-0.239258,-0.146484,-0.734375,0.024017,-0.051636,...,-0.627441,0.0,-0.501953,-0.419434,-0.450928,-0.402588,-0.457764,-0.317139,-0.328369,-0.469482
1,100005,-0.577637,-0.638184,-0.935059,-0.671875,-0.970215,1.079102,-0.464355,-0.891113,-1.172852,...,-0.627441,0.0,-0.501953,0.159180,0.319824,0.256592,0.261719,0.678223,0.364990,0.507324
2,100013,-0.577637,0.313232,0.159546,2.943359,0.247925,-0.127563,-0.916992,-0.886230,0.797852,...,1.593750,0.0,1.992188,-0.373535,-0.314209,-0.475830,-0.317139,-0.541992,-0.171631,-0.358887
3,100028,2.191406,1.346680,2.425781,1.511719,2.806641,0.399658,0.472168,0.221558,0.847656,...,1.593750,0.0,1.992188,-0.455811,-0.475098,-0.337158,-0.486816,-0.424072,-0.434326,-0.504883
4,100038,0.807129,0.106445,0.065796,0.342041,0.235718,-0.783203,0.686523,0.082153,0.279785,...,-0.627441,0.0,-0.501953,0.646973,0.901855,1.006836,0.754395,1.230469,1.196289,1.279297


In [72]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [73]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [74]:
### training keras
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1b = Model1Xgb()
pred_train_1b, pred_test_1b = predict_cv(model_1b, x_train, y_train, x_test)

[00:28:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "num_round", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.78786	eval-auc:0.78658
[1]	train-auc:0.79153	eval-auc:0.78966
[2]	train-auc:0.79177	eval-auc:0.78969
[3]	train-auc:0.79465	eval-auc:0.79184
[4]	train-auc:0.79512	eval-auc:0.79220
[5]	train-auc:0.79623	eval-auc:0.79300
[6]	train-auc:0.79676	eval-auc:0.79327
[7]	train-auc:0.79700	eval-auc:0.79358
[8]	train-auc:0.79716	eval-auc:0.79377
[9]	train-auc:0.79737	eval-auc:0.79394
[10]	train-auc:0.79769	eval-auc:0.79411
[11]	train-auc:0.79771	eval-auc:0.79415
[12]	train-auc:0.79798	eval-auc:0.79448
[13]	train-auc:0.79803	eval-auc:0.79441
[14

[191]	train-auc:0.81053	eval-auc:0.79541
[192]	train-auc:0.81063	eval-auc:0.79541
[193]	train-auc:0.81074	eval-auc:0.79543
[194]	train-auc:0.81090	eval-auc:0.79542
[195]	train-auc:0.81106	eval-auc:0.79545
[196]	train-auc:0.81116	eval-auc:0.79545
[197]	train-auc:0.81128	eval-auc:0.79544
[198]	train-auc:0.81137	eval-auc:0.79546
[199]	train-auc:0.81146	eval-auc:0.79544
[200]	train-auc:0.81154	eval-auc:0.79544
[201]	train-auc:0.81166	eval-auc:0.79543
[202]	train-auc:0.81179	eval-auc:0.79543
[203]	train-auc:0.81185	eval-auc:0.79545
[204]	train-auc:0.81195	eval-auc:0.79543
[205]	train-auc:0.81206	eval-auc:0.79543
[206]	train-auc:0.81219	eval-auc:0.79543
[207]	train-auc:0.81226	eval-auc:0.79543
[208]	train-auc:0.81237	eval-auc:0.79542
[209]	train-auc:0.81258	eval-auc:0.79542
[210]	train-auc:0.81265	eval-auc:0.79542
[211]	train-auc:0.81277	eval-auc:0.79541
[212]	train-auc:0.81286	eval-auc:0.79541
[213]	train-auc:0.81300	eval-auc:0.79543
[214]	train-auc:0.81319	eval-auc:0.79543
[215]	train-auc:

[391]	train-auc:0.84179	eval-auc:0.79512
[392]	train-auc:0.84199	eval-auc:0.79513
[393]	train-auc:0.84222	eval-auc:0.79513
[394]	train-auc:0.84241	eval-auc:0.79513
[395]	train-auc:0.84254	eval-auc:0.79512
[396]	train-auc:0.84270	eval-auc:0.79511
[397]	train-auc:0.84288	eval-auc:0.79510
[398]	train-auc:0.84307	eval-auc:0.79509
[399]	train-auc:0.84327	eval-auc:0.79510
[400]	train-auc:0.84344	eval-auc:0.79511
[401]	train-auc:0.84361	eval-auc:0.79509
[402]	train-auc:0.84380	eval-auc:0.79509
[403]	train-auc:0.84397	eval-auc:0.79508
[404]	train-auc:0.84417	eval-auc:0.79506
[405]	train-auc:0.84432	eval-auc:0.79506
[406]	train-auc:0.84440	eval-auc:0.79506
[407]	train-auc:0.84461	eval-auc:0.79505
[408]	train-auc:0.84483	eval-auc:0.79506
[409]	train-auc:0.84501	eval-auc:0.79505
[410]	train-auc:0.84514	eval-auc:0.79505
[411]	train-auc:0.84532	eval-auc:0.79505
[412]	train-auc:0.84547	eval-auc:0.79507
[413]	train-auc:0.84566	eval-auc:0.79506
[414]	train-auc:0.84585	eval-auc:0.79506
[415]	train-auc:

[591]	train-auc:0.87224	eval-auc:0.79466
[592]	train-auc:0.87240	eval-auc:0.79464
[593]	train-auc:0.87245	eval-auc:0.79463
[594]	train-auc:0.87258	eval-auc:0.79462
[595]	train-auc:0.87272	eval-auc:0.79461
[596]	train-auc:0.87284	eval-auc:0.79460
[597]	train-auc:0.87292	eval-auc:0.79461
[598]	train-auc:0.87303	eval-auc:0.79461
[599]	train-auc:0.87319	eval-auc:0.79462
[600]	train-auc:0.87340	eval-auc:0.79461
[601]	train-auc:0.87349	eval-auc:0.79461
[602]	train-auc:0.87361	eval-auc:0.79461
[603]	train-auc:0.87369	eval-auc:0.79461
[604]	train-auc:0.87389	eval-auc:0.79460
[605]	train-auc:0.87401	eval-auc:0.79461
[606]	train-auc:0.87418	eval-auc:0.79460
[607]	train-auc:0.87437	eval-auc:0.79461
[608]	train-auc:0.87452	eval-auc:0.79461
[609]	train-auc:0.87470	eval-auc:0.79461
[610]	train-auc:0.87480	eval-auc:0.79462
[611]	train-auc:0.87496	eval-auc:0.79463
[612]	train-auc:0.87512	eval-auc:0.79462
[613]	train-auc:0.87526	eval-auc:0.79459
[614]	train-auc:0.87540	eval-auc:0.79459
[615]	train-auc:

[791]	train-auc:0.89629	eval-auc:0.79416
[792]	train-auc:0.89643	eval-auc:0.79417
[793]	train-auc:0.89656	eval-auc:0.79417
[794]	train-auc:0.89665	eval-auc:0.79416
[795]	train-auc:0.89670	eval-auc:0.79415
[796]	train-auc:0.89682	eval-auc:0.79415
[797]	train-auc:0.89700	eval-auc:0.79416
[798]	train-auc:0.89712	eval-auc:0.79418
[799]	train-auc:0.89725	eval-auc:0.79417
[800]	train-auc:0.89733	eval-auc:0.79415
[801]	train-auc:0.89751	eval-auc:0.79413
[802]	train-auc:0.89755	eval-auc:0.79414
[803]	train-auc:0.89774	eval-auc:0.79415
[804]	train-auc:0.89782	eval-auc:0.79415
[805]	train-auc:0.89801	eval-auc:0.79415
[806]	train-auc:0.89808	eval-auc:0.79415
[807]	train-auc:0.89820	eval-auc:0.79415
[808]	train-auc:0.89834	eval-auc:0.79415
[809]	train-auc:0.89845	eval-auc:0.79414
[810]	train-auc:0.89857	eval-auc:0.79415
[811]	train-auc:0.89866	eval-auc:0.79415
[812]	train-auc:0.89877	eval-auc:0.79412
[813]	train-auc:0.89884	eval-auc:0.79411
[814]	train-auc:0.89897	eval-auc:0.79410
[815]	train-auc:

...

[161]	train-auc:0.80790	eval-auc:0.79361
[162]	train-auc:0.80801	eval-auc:0.79361
[163]	train-auc:0.80813	eval-auc:0.79360
[164]	train-auc:0.80824	eval-auc:0.79360
[165]	train-auc:0.80837	eval-auc:0.79359
[166]	train-auc:0.80842	eval-auc:0.79360
[167]	train-auc:0.80849	eval-auc:0.79359
[168]	train-auc:0.80859	eval-auc:0.79361
[169]	train-auc:0.80872	eval-auc:0.79360
[170]	train-auc:0.80882	eval-auc:0.79361
[171]	train-auc:0.80891	eval-auc:0.79360
[172]	train-auc:0.80901	eval-auc:0.79360
[173]	train-auc:0.80913	eval-auc:0.79360
[174]	train-auc:0.80925	eval-auc:0.79360
[175]	train-auc:0.80938	eval-auc:0.79360
[176]	train-auc:0.80949	eval-auc:0.79357
[177]	train-auc:0.80963	eval-auc:0.79362
[178]	train-auc:0.80972	eval-auc:0.79363
[179]	train-auc:0.80985	eval-auc:0.79362
[180]	train-auc:0.80991	eval-auc:0.79360
[181]	train-auc:0.81001	eval-auc:0.79362
[182]	train-auc:0.81016	eval-auc:0.79365
[183]	train-auc:0.81027	eval-auc:0.79363
[184]	train-auc:0.81036	eval-auc:0.79364
[185]	train-auc:

[361]	train-auc:0.83856	eval-auc:0.79314
[362]	train-auc:0.83875	eval-auc:0.79314
[363]	train-auc:0.83900	eval-auc:0.79314
[364]	train-auc:0.83917	eval-auc:0.79312
[365]	train-auc:0.83944	eval-auc:0.79312
[366]	train-auc:0.83967	eval-auc:0.79311
[367]	train-auc:0.83986	eval-auc:0.79311
[368]	train-auc:0.84004	eval-auc:0.79310
[369]	train-auc:0.84024	eval-auc:0.79309
[370]	train-auc:0.84044	eval-auc:0.79308
[371]	train-auc:0.84063	eval-auc:0.79306
[372]	train-auc:0.84082	eval-auc:0.79304
[373]	train-auc:0.84102	eval-auc:0.79305
[374]	train-auc:0.84125	eval-auc:0.79307
[375]	train-auc:0.84140	eval-auc:0.79307
[376]	train-auc:0.84161	eval-auc:0.79306
[377]	train-auc:0.84186	eval-auc:0.79306
[378]	train-auc:0.84206	eval-auc:0.79306
[379]	train-auc:0.84226	eval-auc:0.79305
[380]	train-auc:0.84246	eval-auc:0.79303
[381]	train-auc:0.84266	eval-auc:0.79306
[382]	train-auc:0.84285	eval-auc:0.79306
[383]	train-auc:0.84305	eval-auc:0.79304
[384]	train-auc:0.84330	eval-auc:0.79304
[385]	train-auc:

[561]	train-auc:0.87146	eval-auc:0.79267
[562]	train-auc:0.87160	eval-auc:0.79268
[563]	train-auc:0.87173	eval-auc:0.79268
[564]	train-auc:0.87180	eval-auc:0.79269
[565]	train-auc:0.87190	eval-auc:0.79268
[566]	train-auc:0.87207	eval-auc:0.79269
[567]	train-auc:0.87223	eval-auc:0.79269
[568]	train-auc:0.87234	eval-auc:0.79269
[569]	train-auc:0.87239	eval-auc:0.79269
[570]	train-auc:0.87246	eval-auc:0.79268
[571]	train-auc:0.87259	eval-auc:0.79268
[572]	train-auc:0.87269	eval-auc:0.79267
[573]	train-auc:0.87292	eval-auc:0.79269
[574]	train-auc:0.87299	eval-auc:0.79269
[575]	train-auc:0.87308	eval-auc:0.79268
[576]	train-auc:0.87318	eval-auc:0.79269
[577]	train-auc:0.87339	eval-auc:0.79267
[578]	train-auc:0.87350	eval-auc:0.79265
[579]	train-auc:0.87360	eval-auc:0.79265
[580]	train-auc:0.87374	eval-auc:0.79263
[581]	train-auc:0.87385	eval-auc:0.79262
[582]	train-auc:0.87395	eval-auc:0.79263
[583]	train-auc:0.87408	eval-auc:0.79263
[584]	train-auc:0.87417	eval-auc:0.79263
[585]	train-auc:

[761]	train-auc:0.89498	eval-auc:0.79222
[762]	train-auc:0.89504	eval-auc:0.79221
[763]	train-auc:0.89520	eval-auc:0.79223
[764]	train-auc:0.89533	eval-auc:0.79223
[765]	train-auc:0.89548	eval-auc:0.79223
[766]	train-auc:0.89564	eval-auc:0.79221
[767]	train-auc:0.89571	eval-auc:0.79220
[768]	train-auc:0.89581	eval-auc:0.79219
[769]	train-auc:0.89595	eval-auc:0.79219
[770]	train-auc:0.89603	eval-auc:0.79218
[771]	train-auc:0.89610	eval-auc:0.79219
[772]	train-auc:0.89617	eval-auc:0.79219
[773]	train-auc:0.89634	eval-auc:0.79219
[774]	train-auc:0.89642	eval-auc:0.79219
[775]	train-auc:0.89654	eval-auc:0.79217
[776]	train-auc:0.89664	eval-auc:0.79217
[777]	train-auc:0.89674	eval-auc:0.79218
[778]	train-auc:0.89689	eval-auc:0.79216
[779]	train-auc:0.89705	eval-auc:0.79217
[780]	train-auc:0.89713	eval-auc:0.79218
[781]	train-auc:0.89724	eval-auc:0.79218
[782]	train-auc:0.89730	eval-auc:0.79218
[783]	train-auc:0.89748	eval-auc:0.79217
[784]	train-auc:0.89761	eval-auc:0.79218
[785]	train-auc:

[961]	train-auc:0.91632	eval-auc:0.79172
[962]	train-auc:0.91640	eval-auc:0.79171
[963]	train-auc:0.91650	eval-auc:0.79171
[964]	train-auc:0.91662	eval-auc:0.79172
[965]	train-auc:0.91677	eval-auc:0.79171
[966]	train-auc:0.91688	eval-auc:0.79171
[967]	train-auc:0.91700	eval-auc:0.79171
[968]	train-auc:0.91712	eval-auc:0.79170
[969]	train-auc:0.91718	eval-auc:0.79171
[970]	train-auc:0.91726	eval-auc:0.79171
[971]	train-auc:0.91737	eval-auc:0.79170
[972]	train-auc:0.91747	eval-auc:0.79171
[973]	train-auc:0.91763	eval-auc:0.79172
[974]	train-auc:0.91774	eval-auc:0.79171
[975]	train-auc:0.91788	eval-auc:0.79171
[976]	train-auc:0.91799	eval-auc:0.79170
[977]	train-auc:0.91814	eval-auc:0.79169
[978]	train-auc:0.91822	eval-auc:0.79169
[979]	train-auc:0.91832	eval-auc:0.79168
[980]	train-auc:0.91848	eval-auc:0.79167
[981]	train-auc:0.91857	eval-auc:0.79167
[982]	train-auc:0.91866	eval-auc:0.79167
[983]	train-auc:0.91876	eval-auc:0.79168
[984]	train-auc:0.91887	eval-auc:0.79167
[985]	train-auc:

In [75]:
pred_test_1b

array([0.03276221, 0.10062014, 0.0449992 , ..., 0.00641528, 0.02612578,
       0.20903048], dtype=float32)

In [76]:
pred_train_1b

array([0.23485602, 0.02306518, 0.03871921, ..., 0.03283007, 0.11954995,
       0.08410024], dtype=float32)

In [77]:
pred_1b = []
for a in np.array(pred_train_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

307506

In [78]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'setB1x': pred_1b})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,setB1x
0,100002,0.234856
1,100003,0.023065
2,100004,0.038719
3,100006,0.034008
4,100007,0.027813


In [79]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_setB1xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [80]:
pred_1b = []
for a in np.array(pred_test_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

48744

In [81]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'setB1x': pred_1b})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [82]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_setB1xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)